<a href="https://colab.research.google.com/github/KimNikita/machine-learning-practice/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import pandas as pd

In [34]:
url = "https://raw.githubusercontent.com/KimNikita/machine-learning-practice/main/anime-dataset-2023.csv"
data_raw = pd.read_csv(url)

In [35]:
data_raw.shape

(24905, 24)

In [36]:
data_raw.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


#Удаление ненужных столбцов

In [85]:
data = data_raw.drop(data_raw.columns[[0, 1, 3, 6, 9, 12, 13, 14, 16, 18, 19, 20, 22, 23]], axis=1)
data.head()

,English name,Score,Genres,Type,Episodes,Premiered,Status,Source,Rating,Scored By
0,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi",TV,26.0,spring 1998,Finished Airing,Original,R - 17+ (violence & profanity),914193.0
1,Cowboy Bebop: The Movie,8.38,"Action, Sci-Fi",Movie,1.0,UNKNOWN,Finished Airing,Original,R - 17+ (violence & profanity),206248.0
2,Trigun,8.22,"Action, Adventure, Sci-Fi",TV,26.0,spring 1998,Finished Airing,Manga,PG-13 - Teens 13 or older,356739.0
3,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",TV,26.0,summer 2002,Finished Airing,Original,PG-13 - Teens 13 or older,42829.0
4,Beet the Vandel Buster,6.94,"Adventure, Fantasy, Supernatural",TV,52.0,fall 2004,Finished Airing,Manga,PG - Children,6413.0


#Удаление ненужных строк (фильтрация)

In [86]:
data = data.loc[data['Score'] != 'UNKNOWN']
data = data.loc[data['Episodes'] != '1.0']
data = data.loc[data['Type'].isin(['ONA', 'TV', 'UNKNOWN'])]
data = data.loc[data['Status'].isin(['Finished Airing','UNKNOWN'])]
data = data.loc[data['Source'].isin(['Original', 'Manga' 'Light novel', '4-koma manga', 'Visual novel', 'Other', 'Novel', 'Game', 'Book', 'Web manga', 'Mixed media', 'Card game', 'Web novel'])]
data = data.loc[data['Rating'] != 'UNKNOWN']

data.drop('Status', inplace=True, axis=1)

## Удаление "плохих" жанров

In [87]:
data = data.loc[data['Genres'] != 'UNKNOWN']
length = data.shape[0]
cid = data.columns.get_loc('Genres')
rows_to_drop=[]
for i in range(length):
  cur = data.iat[i, cid]
  if cur.find('Hentai') != -1 or cur.find('Erotica') != -1:
    rows_to_drop.append(i)

data.reset_index(inplace=True)
data.drop(rows_to_drop, inplace=True)
data.drop('index', inplace=True, axis=1)
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)

## Удаление сиквелов \\ приквелов

In [95]:
def mycmp(name):
  return len(name[0])

names=[]
i=0
for name in data['English name']:
  names.append((name, i))
  i+=1
names = sorted(names, key=mycmp)

length = data.shape[0]
rows_to_drop=[]
for index, name in enumerate(names):
  spaces = name[0].count(' ')
  for i in range(index+1, length):
    if names[i][0].count(' ') > spaces and names[i][0].startswith(name[0]):
      rows_to_drop.append(names[i][1])

data.drop(rows_to_drop, inplace=True)
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)

[('K', 1038), ('Air', 28), ('Bug', 2168), ('Noir', 76), ('Kiba', 205), ('Ozma', 1003), ('ID-0', 1587), ('18if', 1711), ('Kino', 1767), ('Rift', 2604), ('Kanon', 313), ('Kaiba', 623), ('K-ON!', 764), ('X-Men', 828), ('Blade', 829), ('No. 6', 922), ('Un-Go', 959), ('Aiura', 1092), ('Norn9', 1392), ('Fafner', 11), ('Blood+', 44), ('Arjuna', 199), ('Shinzo', 261), ('Wowser', 530), ('Catman', 690), ('Canaan', 749), ('11eyes', 816), ('Hyouka', 993), ('ReLIFE', 1325), ('Island', 1519), ('Seiren', 1591), ('Onihei', 1606), ('DIVE!!', 1657), ('Dappys', 1735), ('Minion', 1923), ('ENDRO!', 1994), ('Yasuke', 2387), ('UNKNOWN', 5), ('UNKNOWN', 6), ('UNKNOWN', 10), ('My-Hime', 26), ('UNKNOWN', 33), ('UNKNOWN', 35), ('UNKNOWN', 37), ('UNKNOWN', 38), ('UNKNOWN', 40), ('UNKNOWN', 45), ('UNKNOWN', 48), ('UNKNOWN', 53), ('UNKNOWN', 57), ('UNKNOWN', 62), ('UNKNOWN', 66), ('UNKNOWN', 72), ('UNKNOWN', 75), ('UNKNOWN', 79), ('UNKNOWN', 82), ('UNKNOWN', 85), ('UNKNOWN', 86), ('UNKNOWN', 92), ('UNKNOWN', 94), (

"length = len(names)\nrows_to_drop=[]\nfor index, name in enumerate(names):\n  spaces = name.count(' ')\n  for i in range(index+1, length):\n    if names[i].count(' ') > spaces and names[i].startswith(name):\n      rows_to_drop.append(i)\n\ndata.drop(rows_to_drop, inplace=True)\ndata.drop('index', inplace=True, axis=1)"

#Преобразование данных

## Столбец Genres

In [ ]:
# TODO разбиение\преобразование


## Столбец Episodes

In [55]:
length = data.shape[0]
cid = data.columns.get_loc('Episodes')
for i in range(length):
  cur = data.iat[i, cid]
  if cur != 'UNKNOWN':
    cur = float(cur)
    if cur <= 18:
      data.iat[i, cid] = 'half-season'
    elif 18 < cur <= 36:
      data.iat[i, cid] = 'season'
    elif 36 < cur <= 60:
      data.iat[i, cid] = 'double-season'
    elif 60 < cur:
      data.iat[i, cid] = 'no-season'

## Столбец Premiered

In [56]:
length = data.shape[0]
cid = data.columns.get_loc('Premiered')
for i in range(length):
  cur = data.iat[i, cid]
  if cur.startswith('spring'):
    data.iat[i, cid] = 'spring'
  elif cur.startswith('summer'):
    data.iat[i, cid] = 'summer'
  elif cur.startswith('fall'):
    data.iat[i, cid] = 'fall'
  elif cur.startswith('winter'):
    data.iat[i, cid] = 'winter'


## Калибровка Score с учетом Score By



In [ ]:
data_scored = data['Scored By'].astype('float')
data_scored.quantile([0.005,.01,.05,.1,.15,.2,.25,.3,.35,.4,.45,.5,.55,.6,.65,.7,.75,.8,.85,.9,.95,.99,.995])

In [ ]:
# TODO калибровка
#data.drop('Scored By', inplace=True, axis=1)

# Debug

In [ ]:
print(data['Genres'].unique())

In [57]:
data.shape

(2440, 9)

In [58]:
data.head()

,English name,Score,Genres,Type,Episodes,Premiered,Source,Rating,Scored By
0,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi",TV,season,spring,Original,R - 17+ (violence & profanity),914193.0
1,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",TV,season,summer,Original,PG-13 - Teens 13 or older,42829.0
2,Texhnolyze,7.76,"Action, Drama, Sci-Fi",TV,season,spring,Original,R+ - Mild Nudity,54857.0
3,Neon Genesis Evangelion,8.35,"Action, Avant Garde, Award Winning, Drama, Sci...",TV,season,fall,Original,PG-13 - Teens 13 or older,1024927.0
4,.hack//Sign,6.95,"Adventure, Fantasy, Mystery",TV,season,spring,Original,PG-13 - Teens 13 or older,83009.0


# Борьба с выбросами

In [ ]:
# TODO Genres

print(data['Type'].value_counts())
print(data['Episodes'].value_counts())
print(data['Premiered'].value_counts())
print(data['Source'].value_counts())
print(data['Rating'].value_counts())

In [ ]:
data=data.loc[data['Episodes'] != 'no-season']
data=data.loc[data['Source'].isin(['Original', 'Game', 'Novel', 'Web manga', 'Other', '4-koma manga', 'Visual novel'])]
data=data.loc[data['Rating'] != 'R+ - Mild Nudity']
data.shape

# Заполнение UNKNOWN (пропущенные значения)

In [ ]:
print(data.loc[data['Premiered'] == 'UNKNOWN'])

In [ ]:
# TODO пропорциональное случайное заполнение?

# Преобразование типов

In [ ]:
data['Score'] = data['Score'].astype('float')
# TODO Genres

data['Type'] = data['Type'].astype('category')
data['Episodes'] = data['Episodes'].astype('category')
data['Premiered'] = data['Premiered'].astype('category')
data['Source'] = data['Source'].astype('category')
data['Rating'] = data['Rating'].astype('category')

# TODO
## Разбить значения в Genres (вопрос преподу)
## Провести калибровку Score с учетом Scored By (гугл)
## Борьба с выбросами (лекция)
## Заполнение UNKNOWN (вопрос преподу)
